In [1]:
import os
os.chdir('../../')
from time import time 
import DeepPurpose.models as models
from DeepPurpose.utils import *
from DeepPurpose.dataset import *

RDKit WARNING: [11:35:11] Enabling RDKit 2019.09.3 jupyter extensions


In [6]:
t1 = time()
X_drug, X_target, y = load_process_DAVIS('./data/', binary=False)

drug_encoding = 'Transformer'
target_encoding = 'CNN'
train, val, test = data_process(X_drug, X_target, y, 
                                drug_encoding, target_encoding, 
                                split_method='random',frac=[0.7,0.1,0.2])

# use the parameters setting provided in the paper: https://arxiv.org/pdf/1908.06760.pdf
config = generate_config(drug_encoding = drug_encoding, 
                         target_encoding = target_encoding, 
                         cls_hidden_dims = [1024,512], 
                         train_epoch = 60, 
                         LR = 0.0001, 
                         batch_size = 512,
                         hidden_dim_drug = 128,
                         transformer_emb_size_drug = 128,
                         transformer_intermediate_size_drug = 512,
                         transformer_num_attention_heads_drug = 8,
                         transformer_n_layer_drug = 8,
                         transformer_emb_size_target = 128,
                         transformer_intermediate_size_target = 512,
                         transformer_num_attention_heads_target = 8,
                         transformer_n_layer_target = 4,
                         transformer_dropout_rate = 0.1,
                         transformer_attention_probs_dropout = 0.1,
                         transformer_hidden_dropout_rate = 0.1,
                         cnn_target_filters = [32,64,96],
                         cnn_target_kernels = [4,8,12]
                        )
t2 = time()
print("cost " + str(t2-t1) + " seconds") ### around 3 min. 

Beginning Processing...
Beginning to extract zip file...
Default set to logspace (nM -> p) for easier regression
Done!
in total: 30056 drug-target pairs
encoding drug...
unique drugs: 68
drug encoding finished...
encoding protein...
unique target sequence: 379
protein encoding finished...
splitting dataset...
Done.
cost 156.14085602760315 seconds


In [ ]:
model = models.model_initialize(**config)
model.train(train, val, test)

Let's use CPU/s!
--- Data Preparation ---
--- Go for Training ---
Training at Epoch 1 iteration 0 with loss 30.360344. Total time 0.02361 hours
Validation at Epoch 1 , MSE: 1.0401720376902837 , Pearson Correlation: 0.04092604504404735 with p-value: 0.024841467398258498 , Concordance Index: 0.5097819366734067
Training at Epoch 2 iteration 0 with loss 1.1943979. Total time 0.97944 hours
Validation at Epoch 2 , MSE: 0.8007427858557319 , Pearson Correlation: 0.264268524726774 with p-value: 3.267496720608127e-49 , Concordance Index: 0.6458549046096709
Training at Epoch 3 iteration 0 with loss 0.8310931. Total time 1.83083 hours
Validation at Epoch 3 , MSE: 0.7930355333814978 , Pearson Correlation: 0.339053546099555 with p-value: 9.374325284295059e-82 , Concordance Index: 0.6786436113451896
Training at Epoch 4 iteration 0 with loss 0.93354917. Total time 2.85027 hours
Validation at Epoch 4 , MSE: 0.7777096431635118 , Pearson Correlation: 0.36238530169938044 with p-value: 5.86210407159195e-94

In [4]:
model.save_model('./model_MTDTI_Davis')